In [3]:
import pandas as pd
from utility.processing import processer
import nltk

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 100)

# preprocessing

### data load

In [2]:
train = pd.read_csv('./data/train.csv').fillna('')
test = pd.read_csv('./data/test.csv').fillna('')
df_apply_func = processer.df_apply_func

In [8]:
train.iloc[:,:-1].head()

,id,query,product_title,product_description,median_relevance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,"Red satin accent pillow embroidered with a heart in black thread. 8"" x 8"".",1
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Christmas Lights - Clear Wire,"Set of 10 Battery Operated Train Christmas Lights Item #X124210 Features: Color: multi-color bulbs with matching train light covers / clear wire Multi-color consists of red, green, blue and yellow bulbs Number of bulbs on string: 10 Bulb size: micro LED Spacing between bulbs: 6 inches Lighted length: 4.5 feet Total length: 5.5 feet 12 inch lead cord Additional product features: LED lights use 90% less energy Cool to the touch If one bulb burns out, the rest will stay lit Lights are equipped with Lamp Lock feature, which makes them replaceable, interchangeable and keeps them from falling out Requires 3 ""AA"" batteries (not included) Convenient on/off/timer switch located on battery pack Timer function on battery pack allows for 6 hours on and 18 hours off Cannot connect multiple sets together UL listed for indoor use only Train dimensions: 1.5""H x 1.8""W x .5""D Material(s): plastic/wire/acrylic",4
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4
3,5,wine rack,"Concept Housewares WR-44526 Solid-Wood Ceiling/Wall-Mount Wine Rack, Charcoal Grey, 6 Bottle","Like a silent and sturdy tree, the Southern Enterprises Bird and Branch Coat Rack is an eye-catching addition to your home décor. This tree themed coat rack features strong branches with pinecone accents and a small bird perched at the top to give it a whimsical and welcoming appearance while still making it sturdy enough to hold your coats, hats, umbrellas and more. Whether it serves as a coat rack, a hat rack or a combination of the two, it’ll be a great space saver that gets appreciated for its graceful appearance.\nNumber of Hooks: 10\nFrame Material: Metal\nHardware Material: Metal\nDimensions: 69.5 "" H x 13.25 "" W x 13.25 "" D\nWeight: 12.0 Lb.\nAssembly Details: assembly required",4
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),"WTGR1011\nFeatures\nNickel base, 60,000 average hours, acrylic resin bulb material\nChristmas light bulb\nSteady dimmable replacement lamps\nNickel bases prevent corrosion in sockets\nWattage: 0.96 Watts\nVoltage: 130 Volts\nDimmable: Yes\nLight Source: LED\nBulb Shape Type: Candle\n\nColor Amber\nBulb Color: Amber",2


In [6]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,"Red satin accent pillow embroidered with a heart in black thread. 8"" x 8"".",1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Christmas Lights - Clear Wire,"Set of 10 Battery Operated Train Christmas Lights Item #X124210 Features: Color: multi-color bulbs with matching train light covers / clear wire Multi-color consists of red, green, blue and yellow bulbs Number of bulbs on string: 10 Bulb size: micro LED Spacing between bulbs: 6 inches Lighted length: 4.5 feet Total length: 5.5 feet 12 inch lead cord Additional product features: LED lights use 90% less energy Cool to the touch If one bulb burns out, the rest will stay lit Lights are equipped with Lamp Lock feature, which makes them replaceable, interchangeable and keeps them from falling out Requires 3 ""AA"" batteries (not included) Convenient on/off/timer switch located on battery pack Timer function on battery pack allows for 6 hours on and 18 hours off Cannot connect multiple sets together UL listed for indoor use only Train dimensions: 1.5""H x 1.8""W x .5""D Material(s): plastic/wire/acrylic",4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471
3,5,wine rack,"Concept Housewares WR-44526 Solid-Wood Ceiling/Wall-Mount Wine Rack, Charcoal Grey, 6 Bottle","Like a silent and sturdy tree, the Southern Enterprises Bird and Branch Coat Rack is an eye-catching addition to your home décor. This tree themed coat rack features strong branches with pinecone accents and a small bird perched at the top to give it a whimsical and welcoming appearance while still making it sturdy enough to hold your coats, hats, umbrellas and more. Whether it serves as a coat rack, a hat rack or a combination of the two, it’ll be a great space saver that gets appreciated for its graceful appearance.\nNumber of Hooks: 10\nFrame Material: Metal\nHardware Material: Metal\nDimensions: 69.5 "" H x 13.25 "" W x 13.25 "" D\nWeight: 12.0 Lb.\nAssembly Details: assembly required",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),"WTGR1011\nFeatures\nNickel base, 60,000 average hours, acrylic resin bulb material\nChristmas light bulb\nSteady dimmable replacement lamps\nNickel bases prevent corrosion in sockets\nWattage: 0.96 Watts\nVoltage: 130 Volts\nDimmable: Yes\nLight Source: LED\nBulb Shape Type: Candle\n\nColor Amber\nBulb Color: Amber",2,0.471


### 대/소문자 통합 LED, led, PS2, ps2, levis, Levis, etc...

In [3]:
train = df_apply_func(train, processer.string_lower, first_run=True)
test = df_apply_func(test, processer.string_lower, first_run=True)

In [4]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_preprocessed,product_title_preprocessed,product_description_preprocessed
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decorations,accent pillow with heart design - red/black,red satin accent pillow embroidered with a hea...
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christmas lights,set of 10 battery operated multi led train chr...,set of 10 battery operated train christmas lig...
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,viewsonic pro8200 dlp multimedia projector,
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,concept housewares wr-44526 solid-wood ceiling...,"like a silent and sturdy tree, the southern en..."
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,wintergreen lighting christmas led light bulb ...,"wtgr1011\nfeatures\nnickel base, 60,000 averag..."


### 여러가지 pattern 제거

In [5]:
train = df_apply_func(train, processer.remove_pattern)
test = df_apply_func(test, processer.remove_pattern)

In [6]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_preprocessed,product_title_preprocessed,product_description_preprocessed
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decorations,accent pillow with heart design - red/black,red satin accent pillow embroidered with a hea...
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christmas lights,set of 10 battery operated multi led train chr...,set of 10 battery operated train christmas lig...
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,viewsonic pro8200 dlp multimedia projector,
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,concept housewares wr-44526 solid-wood ceiling...,"like a silent and sturdy tree, the southern en..."
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,wintergreen lighting christmas led light bulb ...,"wtgr1011\nfeatures\nnickel base, 60,000 averag..."


### 단어 양끝 특수기호(구두점) 제거

In [7]:
train = df_apply_func(train, processer.punct)
test = df_apply_func(test, processer.punct)

In [8]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_preprocessed,product_title_preprocessed,product_description_preprocessed
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decorations,accent pillow with heart design red black,red satin accent pillow embroidered with a hea...
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christmas lights,set of 10 battery operated multi led train chr...,set of 10 battery operated train christmas lig...
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,viewsonic pro8200 dlp multimedia projector,
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,concept housewares wr 44526 solid wood ceiling...,like a silent and sturdy tree the southern en...
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,wintergreen lighting christmas led light bulb ...,wtgr1011\nfeatures\nnickel base 60 000 averag...


### Tokenization

In [9]:
train = df_apply_func(train, processer.tokenizer)
test = df_apply_func(test, processer.tokenizer)

In [10]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_preprocessed,product_title_preprocessed,product_description_preprocessed
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decorations,accent pillow with heart design red black,red satin accent pillow embroidered with a hea...
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christmas lights,set of 10 battery operated multi led train chr...,set of 10 battery operated train christmas lig...
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,viewsonic pro8200 dlp multimedia projector,
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,concept housewares wr 44526 solid wood ceiling...,like a silent and sturdy tree the southern ent...
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,wintergreen lighting christmas led light bulb ...,wtgr1011 features nickel base 60 000 average h...


#### 어간, 표제어 추출전 불용어 제거

In [11]:
train = df_apply_func(train, processer.remove_sw)
test = df_apply_func(test, processer.remove_sw)

### 어간추출

In [12]:
train = df_apply_func(train, processer.P_stemmer)
test = df_apply_func(test, processer.P_stemmer)

In [13]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_preprocessed,product_title_preprocessed,product_description_preprocessed
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decor,accent pillow heart design red black,red satin accent pillow embroid heart black th...
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christma light,set 10 batteri oper multi led train christma l...,set 10 batteri oper train christma light item ...
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,viewson pro8200 dlp multimedia projector,
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,concept housewar wr 44526 solid wood ceil wall...,like silent sturdi tree southern enterpris bir...
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,wintergreen light christma led light bulb pack 25,wtgr1011 featur nickel base 60 000 averag hour...


### 표제어 추출(Lemmatization)

In [14]:
train = df_apply_func(train, processer.lemmatizer)
test = df_apply_func(test, processer.lemmatizer)

In [15]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_preprocessed,product_title_preprocessed,product_description_preprocessed
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decor,accent pillow heart design red black,red satin accent pillow embroid heart black th...
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christma light,set 10 batteri oper multi led train christma l...,set 10 batteri oper train christma light item ...
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,viewson pro8200 dlp multimedia projector,
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,concept housewar wr 44526 solid wood ceil wall...,like silent sturdi tree southern enterpris bir...
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,wintergreen light christma led light bulb pack 25,wtgr1011 featur nickel base 60 000 averag hour...


### 어간, 표제어 추출후 불용어 제거

In [16]:
train = df_apply_func(train, processer.remove_sw)
test = df_apply_func(test, processer.remove_sw)

In [17]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_preprocessed,product_title_preprocessed,product_description_preprocessed
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decor,accent pillow heart design red black,red satin accent pillow embroid heart black th...
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christma light,set 10 batteri oper multi led train christma l...,set 10 batteri oper train christma light item ...
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,viewson pro8200 dlp multimedia projector,
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,concept housewar wr 44526 solid wood ceil wall...,like silent sturdi tree southern enterpris bir...
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,wintergreen light christma led light bulb pack 25,wtgr1011 featur nickel base 60 000 averag hour...


### 전처리된 데이터 저장

In [ ]:
# train.to_csv('./data/preprocessed_train.csv', index=False)
# test.to_csv('./data/preprocessed_test.csv', index=False)

In [20]:
from scipy.stats import truncnorm
def get_truncated_normal_sample(mean=2, sd=0):
    if sd == 0:
        sample = np.array([mean]*3)
    else:
        sample = np.round(truncnorm((1 - mean) / sd, (4 - mean) / sd, loc=mean, scale=sd).rvs(3))
        
    if np.median(sample) != mean:
        sample = get_truncated_normal_sample(mean, sd)
    
    return sample

In [21]:
train = train[['query_preprocessed', 'product_title_preprocessed', 'product_description_preprocessed', 'median_relevance', 'relevance_variance']]
test = test[['id', 'query_preprocessed', 'product_title_preprocessed', 'product_description_preprocessed']]

In [22]:
train['sampling'] = train.apply(lambda x: get_truncated_normal_sample(x['median_relevance'], x['relevance_variance']), axis=1)
train = train[['query_preprocessed','product_title_preprocessed','sampling']]

In [23]:
train = pd.DataFrame({'query_preprocessed' : np.repeat(train['query_preprocessed'],3),\
                      'product_title_preprocessed' : np.repeat(train['product_title_preprocessed'],3),\
                      'median_relevance':np.concatenate(train['sampling'].values)})

In [26]:
train.to_csv('./data/preprocessed_train1.csv', index=False)
test.to_csv('./data/preprocessed_test1.csv', index=False)